In [1]:
import panel as pn

pn.extension('tabulator', sizing_mode="stretch_width")

In [2]:
import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [3]:
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
environment()

'notebook'

In [4]:
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]
pn.Row(
    pn.layout.HSpacer(height=50, background=PALETTE[0]),
    pn.layout.HSpacer(height=50, background=PALETTE[1]),
    pn.layout.HSpacer(height=50, background=PALETTE[2]),
)

C:\Users\Luan Lima\AppData\Local\Temp\ipykernel_22348\1656622996.py:3: PanelDeprecationWarning: "HSpacer(..., background='#ff6f69')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#ff6f69'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[0]),
C:\Users\Luan Lima\AppData\Local\Temp\ipykernel_22348\1656622996.py:4: PanelDeprecationWarning: "HSpacer(..., background='#ffcc5c')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#ffcc5c'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[1]),
C:\Users\Luan Lima\AppData\Local\Temp\ipykernel_22348\1656622996.py:5: PanelDeprecationWarning: "HSpacer(..., background='#88d8b0')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#88d8b0'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[2]),


Row(sizing_mode='stretch_width')
    [0] HSpacer(background='#ff6f69', height=50, styles={'background': '#ff6f69'})
    [1] HSpacer(background='#ffcc5c', height=50, styles={'background': '#ffcc5c'})
    [2] HSpacer(background='#88d8b0', height=50, styles={'background': '#88d8b0'})

In [5]:
from bokeh.sampledata.autompg import autompg_clean as df
df.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,mfr
0,18.0,8,307.0,130,3504,12.0,70,North America,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,North America,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,North America,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,North America,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,North America,ford torino,ford


In [6]:
(
    df[
        (df.cyl == 4) & 
        (df.mfr.isin(['ford','chevrolet']))
    ]
    .groupby(['origin', 'cyl', 'mfr', 'yr'])['hp'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='yr')
).head(1)

,origin,cyl,mfr,yr,hp
0,North America,4,chevrolet,71,81.0


In [7]:
idf = df.interactive()

In [9]:
cylinders = pn.widgets.IntSlider(name='Cylinders', start=4, end=8, step=2)
mfr = pn.widgets.ToggleGroup(
    name='MFR',
    options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'], 
    value=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    button_type='success')
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['hp', 'weight'],
    button_type='success'
)

In [10]:
ipipeline = (
    idf[
        (idf.cyl == cylinders) & 
        (idf.mfr.isin(mfr))
    ]
    .groupby(['origin', 'mpg'])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='mpg')  
    .reset_index(drop=True)
)
ipipeline.head()

In [11]:
if environment()=="server":
   theme="fast"
else:
   theme="simple"

In [12]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10)
itable

In [13]:
ihvplot = ipipeline.hvplot(x='mpg', y=yaxis, by='origin', color=PALETTE, line_width=6, height=400)
ihvplot

In [14]:
template = pn.template.FastListTemplate(
    title='Interactive DataFrame Dashboards with hvplot .interactive', 
    sidebar=[cylinders, 'Manufacturers', mfr, 'Y axis' , yaxis],
    main=[ihvplot.panel(), itable.panel()],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();